In [2]:
import osmnx as ox
import time
from shapely.geometry import Polygon
import os
import csv
import numpy as np


In [3]:
def save_graph_shapefile_directional(name_file, G, filepath=None, encoding="utf-8"):
    # default filepath if none was provided
    if filepath is None:
        filepath = os.path.join(ox.settings.data_folder, "graph_shapefile")

    # if save folder does not already exist, create it (shapefiles
    # get saved as set of files)
    if not filepath == "" and not os.path.exists(filepath):
        os.makedirs(filepath)
    filepath_nodes = os.path.join(filepath, "nodes.shp")
    filepath_edges = os.path.join(filepath, "edges.shp")


    # convert undirected graph to gdfs and stringify non-numeric columns
    gdf_nodes, gdf_edges = ox.utils_graph.graph_to_gdfs(G)
    gdf_nodes = ox.io._stringify_nonnumeric_cols(gdf_nodes)
    gdf_edges = ox.io._stringify_nonnumeric_cols(gdf_edges)
    # We need an unique ID for each edge
    # filtramos calles
    # gdf_edges = gdf_edges[gdf_edges["highway"]!='living_street']
    # gdf_edges = gdf_edges[gdf_edges["highway"]!='residential']
    # gdf_edges = gdf_edges[gdf_edges["highway"]!="['living_street', 'residential']"]
    # gdf_edges = gdf_edges[gdf_edges["highway"]!="['residential', 'living_street']"]
    # ids = [i + 1 for i in range(len(gdf_edges.index))]
    # gdf_edges["id"] = ids
    gdf_edges["fid"] = np.arange(0, gdf_edges.shape[0], dtype='int')

    # save the nodes and edges as separate ESRI shapefiles
    gdf_nodes.to_file(filepath_nodes, encoding=encoding)
    gdf_edges.to_file(filepath_edges, encoding=encoding)

    # a = gdf_nodes.to_json()
    # b = gdf_edges.to_json()
    #
    # print(gdf_nodes.columns)
    # print(gdf_nodes.head())
    # print(gdf_edges.columns)
    # print(gdf_edges.head())

    # archivo nodo to csv
    osmid = [i for i in gdf_nodes.index]
    y = list(gdf_nodes['y'])
    x = list(gdf_nodes['x'])
    street_count = list(gdf_nodes['street_count'])
    highway = list(gdf_nodes['highway'])
    geometry = list(gdf_nodes['geometry'])

    with open(os.path.join("osm/{}/{}_nodos_completo.csv".format(name_file, name_file)), "w", newline="",
              encoding="utf-8") as f:
        writer = csv.writer(f, delimiter=",")
        writer.writerow(["osmid", "y", "x", "street_count", "highway", "geometry"])

        for i in range(len(list(gdf_nodes['y']))):
            writer.writerow([osmid[i], y[i], x[i], street_count[i], highway[i], geometry[i]])

    # archivo arco to csv
    fid = np.arange(0, gdf_edges.shape[0], dtype='int')
    osmid = list(gdf_edges['osmid'])
    u = [u for u, v, key in gdf_edges.index]
    v = [v for u, v, key in gdf_edges.index]
    key = [key for u, v, key in gdf_edges.index]
    highway = list(gdf_edges['highway'])
    oneway = list(gdf_edges['oneway'])
    length = list(gdf_edges['length'])
    geometry = list(gdf_edges['geometry'])
    name = list(gdf_edges['name'])
    lanes = list(gdf_edges['lanes'])
    maxspeed = list(gdf_edges['maxspeed'])

    with open(os.path.join("osm/{}/{}_arcos_completo.csv".format(name_file, name_file)), "w", newline="",
              encoding="utf-8") as f:
        writer = csv.writer(f, delimiter=",")
        writer.writerow(
            ["fid", "osmid", "osmid_nodo_origen", "osmid_nodo_destino", "key", "highway", "oneway", "length",
             "geometry",
             "name", "lanes", "maxspeed"])

        for i in range(len(list(gdf_edges['osmid']))):
            writer.writerow(
                [fid[i], osmid[i], u[i], v[i], key[i], highway[i], oneway[i], length[i], geometry[i], name[i], lanes[i],
                 maxspeed[i]])


In [ ]:
print("osmnx version", ox.__version__)
# Download by a bounding box
x1, y1, x2, y2 = 148.9643, -35.4724, 149.2603, -35.1393 # West/South/East/North
boundary_polygon = Polygon([(x1, y1), (x2, y1), (x2, y2), (x1, y2)])
G = ox.graph_from_polygon(boundary_polygon, network_type='drive')

start_time = time.time()
save_graph_shapefile_directional('Canberra', G, filepath='osm/Canberra')
print("--- %s seconds ---" % (time.time() - start_time))


In [10]:
# Convert the output to simulator format
import pandas as pd
import numpy as np

TARGET_CITY = 'Canberra'
# Load the nodes
nodes = pd.read_csv('osm/{}/{}_nodos_completo.csv'.format(TARGET_CITY, TARGET_CITY))

# Create the nodes df
sim_nodes_df = pd.DataFrame(columns=['node_id', 'longitude', 'latitude', 'zone_id', 'osm_node_id'])

# Load the edges
edges = pd.read_csv('osm/{}/{}_arcos_completo.csv'.format(TARGET_CITY, TARGET_CITY))


In [19]:
# Step 1: Create the nodes
sim_nodes_df['osm_node_id'] = nodes['osmid']
sim_nodes_df['longitude'] = nodes['x']
sim_nodes_df['latitude'] = nodes['y']
sim_nodes_df['node_id'] = np.arange(1, len(sim_nodes_df) + 1)
sim_nodes_df['zone_id'] = 0

sim_nodes_df.head()

,node_id,longitude,latitude,zone_id,osm_node_id
0,1,149.206543,-35.203404,0,770347
1,2,149.214446,-35.199750,0,770350
2,3,149.251382,-35.181436,0,770373
3,4,149.140940,-35.248882,0,7987009
4,5,149.140294,-35.251121,0,7987066


In [24]:
# Step 2: Create the edges
edges.head()

sim_edge_df = pd.DataFrame(columns=['edge_id', 'from_node_id', 'to_node_id', 'length', 'max_speed', 'edge_type', 'oneway', 'osm_way_id'])

sim_edge_df['from_node_id'] = edges['osmid_nodo_origen']
sim_edge_df['to_node_id'] = edges['osmid_nodo_destino']
sim_edge_df['length'] = edges['length']
sim_edge_df['max_speed'] = edges['maxspeed']
sim_edge_df['edge_type'] = edges['highway']
sim_edge_df['oneway'] = edges['oneway']
sim_edge_df['osm_way_id'] = edges['osmid']
sim_edge_df['edge_id'] = np.arange(1, len(sim_edge_df) + 1)

sim_edge_df.head()

,edge_id,from_node_id,to_node_id,length,max_speed,edge_type,oneway,osm_way_id
0,1,770347,903974904,397.268,"['110', '60']",motorway_link,True,"[784540081, 4176451]"
1,2,770347,770350,827.434,110,motorway,True,764315373
2,3,770350,770373,4309.403,110,motorway,True,"[4176400, 28950545, 764315378, 764315380, 7643..."
3,4,770373,30201096,615.955,NaN,motorway_link,True,4734792
4,5,770373,223273148,648.482,110,motorway,True,"[764315393, 764315383]"


In [25]:
# Step 3: Mapping the node_id in edges to the new node_id

sim_edge_df['from_node_id'] = sim_edge_df['from_node_id'].map(sim_nodes_df.set_index('osm_node_id')['node_id'])
sim_edge_df['to_node_id'] = sim_edge_df['to_node_id'].map(sim_nodes_df.set_index('osm_node_id')['node_id'])

sim_edge_df.head()

,edge_id,from_node_id,to_node_id,length,max_speed,edge_type,oneway,osm_way_id
0,1,1,13540,397.268,"['110', '60']",motorway_link,True,"[784540081, 4176451]"
1,2,1,2,827.434,110,motorway,True,764315373
2,3,2,3,4309.403,110,motorway,True,"[4176400, 28950545, 764315378, 764315380, 7643..."
3,4,3,1602,615.955,NaN,motorway_link,True,4734792
4,5,3,8443,648.482,110,motorway,True,"[764315393, 764315383]"


In [26]:
# Step 4: Duplicate the edges which are not one way, with reversed direction
sim_edge_df_reverse = sim_edge_df.copy()
sim_edge_df_reverse['from_node_id'] = sim_edge_df['to_node_id']
sim_edge_df_reverse['to_node_id'] = sim_edge_df['from_node_id']
# delete the oneway edges
sim_edge_df_reverse = sim_edge_df_reverse[sim_edge_df_reverse['oneway'] == False]
# reset the oneway
sim_edge_df_reverse['oneway'] = True
# reset the edge_id
sim_edge_df_reverse['edge_id'] = np.arange(len(sim_edge_df) + 1, len(sim_edge_df) + 1 + len(sim_edge_df_reverse))
# cat the two edges
sim_edge_df = pd.concat([sim_edge_df, sim_edge_df_reverse], axis=0)

sim_edge_df.head()

,edge_id,from_node_id,to_node_id,length,max_speed,edge_type,oneway,osm_way_id
0,1,1,13540,397.268,"['110', '60']",motorway_link,True,"[784540081, 4176451]"
1,2,1,2,827.434,110,motorway,True,764315373
2,3,2,3,4309.403,110,motorway,True,"[4176400, 28950545, 764315378, 764315380, 7643..."
3,4,3,1602,615.955,NaN,motorway_link,True,4734792
4,5,3,8443,648.482,110,motorway,True,"[764315393, 764315383]"


In [32]:
# Step 5: Assign Traverse Time according to edge_type
speed_dict = {
    # 主干道
    'motorway': 110,          # 高速公路
    'motorway_link': 60,      # 高速公路匝道
    'trunk': 80,              # 城市快速路
    'trunk_link': 50,         # 快速路匝道
    
    # 城市主要道路
    'primary': 70,            # 城市主干道
    'primary_link': 50,       # 主干道匝道
    'secondary': 60,          # 城市次干道
    'secondary_link': 40,     # 次干道匝道
    'tertiary': 50,          # 三级道路
    'tertiary_link': 40,     # 三级道路匝道
    
    # 次要道路
    'residential': 40,        # 居住区道路
    'living_street': 30,      # 生活街区
    'unclassified': 40,      # 未分类道路
    'busway': 50,            # 公交专用道
    
    # 组合道路类型(取较高速度限制)
    "['motorway', 'trunk']": 110,
    "['motorway', 'primary']": 110,
    "['primary_link', 'residential']": 50,
    "['living_street', 'tertiary']": 50,
    "['residential', 'unclassified']": 40,
    "['living_street', 'unclassified']": 40,
    "['secondary', 'residential']": 60,
    "['secondary', 'unclassified']": 60,
    "['tertiary', 'residential']": 50,
    "['residential', 'tertiary']": 50,
    "['primary_link', 'secondary_link']": 50,
    "['primary_link', 'unclassified']": 50,
    "['secondary', 'secondary_link']": 60,
    "['secondary', 'tertiary']": 60
}

sim_edge_df['edge_time'] = (sim_edge_df['length']/1000)/sim_edge_df['edge_type'].map(speed_dict)*3600 # in seconds
# round to seconds
sim_edge_df['edge_time'] = sim_edge_df['edge_time'].round(0)
sim_edge_df.head()



,edge_id,from_node_id,to_node_id,length,max_speed,edge_type,oneway,osm_way_id,edge_time
0,1,1,13540,397.268,"['110', '60']",motorway_link,True,"[784540081, 4176451]",24.0
1,2,1,2,827.434,110,motorway,True,764315373,27.0
2,3,2,3,4309.403,110,motorway,True,"[4176400, 28950545, 764315378, 764315380, 7643...",141.0
3,4,3,1602,615.955,NaN,motorway_link,True,4734792,37.0
4,5,3,8443,648.482,110,motorway,True,"[764315393, 764315383]",21.0


In [33]:
# Step 6: Clean the Edge Dataframe
sim_edge_df_clean = sim_edge_df[['edge_id', 'from_node_id', 'to_node_id', 'edge_time']]
sim_edge_df_clean.head()


,edge_id,from_node_id,to_node_id,edge_time
0,1,1,13540,24.0
1,2,1,2,27.0
2,3,2,3,141.0
3,4,3,1602,37.0
4,5,3,8443,21.0


In [ ]:
# Step 7: Find isolated nodes and remove them
sim_nodes_df_clean = sim_nodes_df.copy()
sim_nodes_df_clean = sim_nodes_df_clean[sim_nodes_df_clean['node_id'].isin(sim_edge_df_clean['from_node_id']) | sim_nodes_df_clean['node_id'].isin(sim_edge_df_clean['to_node_id'])]
print("num of isolated nodes: ", len(sim_nodes_df) - len(sim_nodes_df_clean))
sim_nodes_df_clean.head()

num of isolated nodes:  0


,node_id,longitude,latitude,zone_id,osm_node_id
0,1,149.206543,-35.203404,0,770347
1,2,149.214446,-35.199750,0,770350
2,3,149.251382,-35.181436,0,770373
3,4,149.140940,-35.248882,0,7987009
4,5,149.140294,-35.251121,0,7987066


In [42]:
# Step 8: Check if all the nodes in the edge dataframe are in the node dataframe
reduntant_nodes = sim_edge_df_clean[~sim_edge_df_clean['from_node_id'].isin(sim_nodes_df_clean['node_id']) | ~sim_edge_df_clean['to_node_id'].isin(sim_nodes_df_clean['node_id'])]
print("num of reduntant nodes: ", len(reduntant_nodes))

num of reduntant nodes:  0


In [ ]:
# Step 9: Use K-Means to cluster the nodes
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

NUM_ZONES = 32

scaler = StandardScaler()
sim_nodes_df_clean[['longitude', 'latitude']] = scaler.fit_transform(sim_nodes_df_clean[['longitude', 'latitude']])
kmeans = KMeans(n_clusters=NUM_ZONES, random_state=0).fit(sim_nodes_df_clean[['longitude', 'latitude']])
sim_nodes_df_clean['zone_id'] = kmeans.labels_

sim_nodes_df_clean.head()

/Users/ricardo/opt/anaconda3/envs/elispe/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [ ]:
# Step 10: Build Graph, find the shortest path and save all_path_matrix, all_path_time_matrix
import networkx as nx

